In [ ]:
from pykrx import stock

In [ ]:
# 날짜별 주가 데이터 반환 함수
import json

def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

# 삼성전자 종목 확인용
print(get_market_ohlcv("20220720", "20220810", "005930"))

In [4]:
# tools 정의
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


In [ ]:
# openAI API 호출
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [
    {
        "role": "user",
        "content": "LG전자(066570)의 2022년 8월 1일부터 2022년 12월 1일까지의 주가 데이터를 분석해줘."
    }
]

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1024,
    tools=tools,
    tool_choice="auto" if tools is not None else None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response.choices[0].message)

In [ ]:
# openAI API 호출 시 tool_call 사용하여 호출

available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)